In [374]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split#, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from pandas_profiling import ProfileReport
from category_encoders import OneHotEncoder
import ipywidgets

# how to become a superhost:
# https://www.airbnb.com/help/article/829/how-do-i-become-a-       

In [362]:
filepath = 'Data/listings.csv.gz'

def wrangle(filepath):
    air = pd.read_csv(filepath, index_col='id')

    # Dropping columns with too many null values
    drop_cols_null = ['neighbourhood_group_cleansed', 'bathrooms',
                     'calendar_updated']
    # air.isnull().sum()
    air.drop(columns=drop_cols_null, inplace=True)
    
    # Dropping columns that seem irrelevant URL links
    drop_cols = ['listing_url', 'scrape_id', 'picture_url', 'description',
                 'neighborhood_overview', 'host_url', 'host_location', 'name',
                 'host_name', 'host_about', 'host_thumbnail_url',
                 'host_picture_url', 'host_neighbourhood', 'neighbourhood',
                 'host_total_listings_count',
                 'last_scraped']

    # Seem irrelevant/ noise for our data
    drop_cols2 = ['minimum_nights', 'maximum_nights',
                  'minimum_minimum_nights', 'maximum_minimum_nights',
                   'minimum_maximum_nights', 'maximum_maximum_nights',
                   'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
                   'availability_30', 'availability_60', 'availability_90',
                   'availability_365', 'calendar_last_scraped',
                   'calculated_host_listings_count',
                   'calculated_host_listings_count_entire_homes',
                   'calculated_host_listings_count_private_rooms',
                   'calculated_host_listings_count_shared_rooms',
                   'review_scores_accuracy',
                   'review_scores_cleanliness', 'review_scores_checkin',
                   'review_scores_communication', 'review_scores_location',
                   'review_scores_value', 'license', 'number_of_reviews_ltm',
                   'number_of_reviews_l30d', 'first_review', 'last_review', 'host_acceptance_rate',
                   'reviews_per_month',
                   'host_response_time', 'host_response_rate', 'host_since']
#     air.drop(columns=drop_cols, inplace=True)
#     air.drop(columns=drop_cols2, inplace=True)

    # Change the columns t (true) and f (false) to 0 and 1's
    ### note, change type
    
    # Dropping the host_id

    air.loc[air['host_identity_verified'] == "t", 'host_identity_verified'] = 1
    air.loc[air['host_identity_verified'] == "f", 'host_identity_verified'] = 0

    air.loc[air['host_has_profile_pic'] == "t", 'host_has_profile_pic'] = 1
    air.loc[air['host_has_profile_pic'] == "f", 'host_has_profile_pic'] = 0

    air.loc[air['instant_bookable'] == "t", 'instant_bookable'] = 1
    air.loc[air['instant_bookable'] == "f", 'instant_bookable'] = 0

    air['super_host'] = 1
    air.loc[air['host_is_superhost'] == 'f', 'super_host'] = 0
    air.drop(columns='host_is_superhost', inplace=True)

    # cleaning the price column to be integers
    air['price'] = air['price'].str.replace(',', '').str.strip('$').astype(float)

    #if we have time we can review this items:
    cols_stretch_goal = ['host_verifications', 'amenities', 'property_type']
#     air.drop(columns=cols_stretch_goal, inplace=True)

    air.head(20)
    # Let's rename the columns
#     air.columns = ['host_id', 'listing_count', 'profile_pic', 'id_verified',
#                   'ward', 'lat', 'long', 'room_type', 'guest_count', 'bath',
#                   'bedroom', 'beds', 'price', 'review_count', 'review_score',
#                   'ins_book', 'super_host']
    
    return air

In [365]:
air = wrangle(filepath)
air.head(5)

,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,super_host
id,,,,,,,,,,,,,,,,,,,,,
197677,https://www.airbnb.com/rooms/197677,20210327014458,2021-03-28,Oshiage Holiday Apartment,<b>The space</b><br />We are happy to welcome ...,NaN,https://a0.muscache.com/pictures/38437056/d27f...,964081,https://www.airbnb.com/users/show/964081,Yoshimi & Marek,...,9.0,10.0,M130003350,0,1,1,0,0,1.42,0
776070,https://www.airbnb.com/rooms/776070,20210327014458,2021-03-28,Kero-kero house room 1,We have been in airbnb since 2011 and it has g...,We love Nishinippori because is nearer to Toky...,https://a0.muscache.com/pictures/efd9f039-dbd2...,801494,https://www.airbnb.com/users/show/801494,Kei,...,10.0,10.0,M130000243,1,1,0,1,0,2.25,1
899003,https://www.airbnb.com/rooms/899003,20210327014458,2021-03-29,"Classy room @Shinjuku, Takadanoba",..*+;.*:' Popular room in Airbnb@Tokyo *;+:..*...,NaN,https://a0.muscache.com/pictures/20005274/1df3...,4799233,https://www.airbnb.com/users/show/4799233,Yu,...,9.0,9.0,Other reasons | 【マンスリー契約】最低30泊以上の一時使用賃貸借契約を結びま...,0,2,2,0,0,0.94,0
1016831,https://www.airbnb.com/rooms/1016831,20210327014458,2021-03-29,WOMAN ONLY LICENSED ! Cosy & Cat behnd Shibuya,female travellers here only.<br />Sorry no boy...,The location is walkable distance to famous Sh...,https://a0.muscache.com/pictures/20134416/5c34...,5596383,https://www.airbnb.com/users/show/5596383,Wakana,...,10.0,10.0,M130001107,0,1,0,1,0,2.17,0
1033276,https://www.airbnb.com/rooms/1033276,20210327014458,2021-03-29,private room @Senju area,Our house is new and clean <br />Big hub stati...,There are shopping mall near Senjuohashi stati...,https://a0.muscache.com/pictures/71577415/b060...,5686404,https://www.airbnb.com/users/show/5686404,NaN,...,9.0,9.0,M130007760,0,2,0,2,0,0.75,1


In [378]:
profile = ProfileReport(air)

In [379]:
profile

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [303]:
wards = air.ward.unique()
print(wards)

['Sumida Ku' 'Kita Ku' 'Shinjuku Ku' 'Setagaya Ku' 'Adachi Ku'
 'Shibuya Ku' 'Katsushika Ku' 'Meguro Ku' 'Toshima Ku' 'Koto Ku'
 'Shinagawa Ku' 'Minato Ku' 'Suginami Ku' 'Akishima Shi' 'Bunkyo Ku'
 'Taito Ku' 'Itabashi Ku' 'Arakawa Ku' 'Machida Shi' 'Higashimurayama Shi'
 'Chuo Ku' 'Nakano Ku' 'Kokubunji Shi' 'Hino Shi' 'Mitaka Shi'
 'Hachioji Shi' 'Edogawa Ku' 'Musashino Shi' 'Nerima Ku' 'Ota Ku'
 'Akiruno Shi' 'Chiyoda Ku' 'Kodaira Shi' 'Fuchu Shi' 'Fussa Shi'
 'Tama Shi' 'Komae Shi' 'Chofu Shi' 'Koganei Shi' 'Nishitokyo Shi'
 'Higashikurume Shi' 'Hinohara Mura' 'Ome Shi' 'Kunitachi Shi'
 'Okutama Machi' 'Hamura Shi' 'Tachikawa Shi' 'Musashimurayama Shi']


In [279]:
air.columns

Index(['id', 'host_id', 'host_since', 'host_listings_count',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'room_type',
       'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'price',
       'number_of_reviews', 'review_scores_rating', 'instant_bookable',
       'reviews_per_month', 'super_host'],
      dtype='object')

In [300]:
# air.property_type.unique()

In [276]:
air.room_type.unique()

array(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'],
      dtype=object)

In [238]:
# ver_list = []
# for person in verifications:
#     new = person.split(",")
#     for item in new:
#         verification_item =item.replace("'", '').strip("[").strip("]").strip(' ')
#         if verification_item not in ver_list:
#             print(verification_item)
#             ver_list.append(verification_item)
# len(ver_list)

In [138]:
air.number_of_reviews.describe()

count    11197.000000
mean        26.626150
std         41.647206
min          0.000000
25%          1.000000
50%         10.000000
75%         34.000000
max        479.000000
Name: number_of_reviews, dtype: float64

In [266]:
air.instant_bookable.describe()

count     11197
unique        2
top           t
freq       8301
Name: instant_bookable, dtype: object

In [ ]:
# for future reference: data for the station's locations
# https://ckan-tokyochallenge.odpt.org/en/dataset?tags=列車ロケーション情報-train_location_information


In [359]:
target='price'
X = air.drop(columns=target)
y = air[target]

# Let's split into a test and
X_train, X_t, y_train, y_t = train_test_split(X,y, test_size=.2, random_state=7)

# Let's split our test data into validation and test
X_val, X_test, y_val, y_test = train_test_split(X_t,y_t, test_size=.2, random_state=7)

print('Training:', len(X_train))
print('Validation:', len(X_val))
print('Test:', len(X_test))

Training: 8957
Validation: 1792
Test: 448


In [360]:
# Random forest regression

model_rf = make_pipeline(OneHotEncoder(),
                         SimpleImputer(),
                         RandomForestRegressor(random_state=70))
model_rf.fit(X_train,y_train);


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


KeyboardInterrupt: 

In [ ]:
# Mean Absolute Error Accuracy Score
# Training
print('Random Forest Regressor')

y_pred = len(y_train) *[y_train.mean()]
baseline_mae = mean_absolute_error(y_train,y_pred)
print('Baseline MAE: ', baseline_mae)

training_acc = mean_absolute_error(y_train,model_rf.predict(X_train))
print('MAE Train: ', training_acc)

# Validation
val_acc = mean_absolute_error(y_val,model_rf.predict(X_val))
print('MAE Validation: ',val_acc)

# R squared
print('Rsquared Validation:', r2_score(y_val, model_rf.predict(X_val)))

In [ ]:
X_val

In [340]:
# dir(model_rf.named_steps['randomforestregressor'])

In [ ]:
# Create bar chart
# Add coefficients from Model_r's named steps
coefficients = model_rf.named_steps['randomforestregressor'].feature_importances_

# Add feature names from Model_r's named steps
features = model_rf.named_steps['onehotencoder'].get_feature_names()

# Combine the two values into a single object, sorting based on the absolute value
feat_imp = pd.Series(coefficients, index=features).sort_values(key=abs)
# Create the bar chart and limit it to the top 10 features
feat_imp.tail(20).plot(kind='barh')
# Add labels to the chart
plt.xlabel('Coefficient [$]')
plt.ylabel('Feature')
plt.title('Top 10 Coefficients for Ridge Regression \nand Their Impact on Sale Price');

In [346]:
air.host_id.nunique()

2855

In [347]:
len(air)

11197